In [ ]:
!pip install fasttext scikit-learn

import numpy as np
import fasttext
import sklearn
import matplotlib.pyplot as plt

# Document classification using k-NN and Naive Bayes
Today, we will work with the [20 newsgroups](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) dataset from 1997. It contains relatively short articles (as well as some longer ones) from 20 different newsgroups (see below).

Our goal is to decide which group a given document comes from. To make the task more challenging, we will remove any superfluous metadata, and we'll judge each document purely by its content. Don't expect perfect accuracy.

In last week's lab, you were introduced to TF-IDF and sentence embedding using Word2Vec. We'll use these techniques as well but, this time we'll employ k-Nearest Neighbors and Naive Bayes classifiers.



In [ ]:
# fetch dataset
from sklearn.datasets import fetch_20newsgroups
train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

fig, ax = plt.subplots(figsize=(6, 3))
ax.bar(range(len(train.target_names)), np.bincount(train.target), align='center')
plt.xticks(range(len(train.target_names)), train.target_names, rotation=45, ha='right')
plt.show()

# 1) Vectorizing the documents
The sklearn implementations of both k-Nearest Neighbors and Naive Bayes classifiers take vectors of fixed length. So, we must first vectorize the documents.

Sklearn already implements [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), which simply counts the occurrences of each word from a vocabulary, and [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), which you should already be familiar with.

We'll also implement our own vectorizers.
First, implement `EmbeddingVectorizer` below, which will vectorize documents using a given Word2Vec embedding model.

Last week, you saw that ignoring certain "stopwords" often helps. Instead of ignoring them, we could lower their influence by weighting each word embedding by its inverse document frequency (IDF). Try implementing this in `IdfWeightedEmbeddingVectorizer`.

In [ ]:
import os

# download a pre-trained Word2Vec model
if os.path.exists('cc.en.300.bin'):
    print('FastText models already downloaded.')
else:
    ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
    ! gunzip cc.en.300.bin.gz

embeddings = fasttext.load_model('cc.en.300.bin')

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
class EmbeddingVectorizer(TransformerMixin, BaseEstimator):
  def __init__(self, embedding):
    self.embedding: fasttext.FastText._FastText = embedding
    self.dim = embedding.get_dimension()
  def fit(self, X, y=None):
    return self
  def transform(self, X: list[str]) -> np.ndarray:
    # TODO: implement this method. It should:
    # 1. take a list of documents X
    # 2. use self.embedding to convert each document to a vector of fixed size
    # (for example by computing the mean of the word embeddings)
    # 3. return a numpy array
    #
    # hint: self.embedding should already have a method to get vector from a whole sentence
    # 2nd hint: that method does not like newlines, you can remove them with document.replace('\n', ' ')
    raise NotImplementedError()

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
class IdfWeightedEmbeddingVectorizer(TransformerMixin, BaseEstimator):
  def __init__(self, embedding):
    self.embedding: fasttext.FastText._FastText = embedding
    self.idf_weights = None
    self.dim = embedding.get_dimension()

  def fit(self, X, y=None):
    tfidf = TfidfVectorizer(analyzer=lambda x: x)
    tfidf.fit(X)
    # if a word was never seen - it must be at least as infrequent
    # as any of the known words - so the default idf is the max of
    # known idf's
    max_idf = max(tfidf.idf_)
    self.idf_weights = defaultdict(
        lambda: max_idf,
        [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    return self

  def transform(self, X: list[str]) -> np.ndarray:
    # TODO: implement
    # Same as before but this time compute a weighted mean of the word embeddings
    # using the computed IDF weights.
    # Hint: some documents have no words, make sure to return a vector of zeroes of the correct dimension for those
    # Hint 2: you can vectorize a word with self.embedding.get_word_vector(w) and get its weight with self.idf_weights[w]
    raise NotImplementedError()


# 1.5) Simple document lookup engine
Lets use our vectorizers in a very simplistic document lookup engine.
The lookup engine works by comparing the vectors of stored documents with a vector computed from a given query. It then shows the documents whose vectors are closest to the query. Try it out!

What metric makes sense for finding the closest document vectors?

Why doesn't the euclidean metric work well with the TF-IDF vectors?

In [ ]:
from sklearn.metrics import pairwise_distances

class RelevantDocumentFinder:
  def __init__(self, documents: list[str], labels: list[str], vectorizer):
    self.documents = documents
    self.labels = labels
    self.vectorizer = vectorizer
    self.vectors = self.vectorizer.fit_transform(documents)
    pass
  def query(self, query: str, num_nearest: int = 5, metric: str = 'euclidean'):
    query_vec = self.vectorizer.transform([query])
    distances = pairwise_distances(self.vectors, query_vec.reshape(1, -1), metric=metric)
    nearest = np.argsort(distances, axis=0)[:num_nearest].flatten()
    for idx in nearest:
      print(f"Distance: {distances[idx][0]:.2f}")
      print(f"Group: {train.target_names[self.labels[idx]]}")
      print(f"Document:\n{self.documents[idx]}")
      print("-----------------------------------------------")

finder = RelevantDocumentFinder(train.data, train.target, EmbeddingVectorizer(embedding=embeddings))
#finder = RelevantDocumentFinder(train.data, train.target, IdfWeightedEmbeddingVectorizer(embedding=embeddings))
#finder = RelevantDocumentFinder(train.data, train.target, TfidfVectorizer())

query = "The pope and the church."
finder.query(query, metric='cosine')

# 2. Document classification with k-Nearest Neighbors
In `RelevantDocumentFinder`, we looked at which document vectors are close to the vector of our query. We could also use the labels of those nearby documents to decide which newsgroup our query corresponds to.

Treating a previously unseen document as a query, finding its `k` nearest neighboring documents, and taking the most frequent label is precisely how the [`KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) works.

Try it together with different document vectorizers. Which one works best for this dataset? Which metric works best? Also try neighbor weighting.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Build a k-nearest neighbors classifier
# First vectorize the documents. Try different vectorizers:
#   CountVectorizer, TfidfVectorizer, EmbeddingVectorizer, IdfWeightedEmbeddingVectorizer
# Try different parameters for KNeighborsClassifier, which metric works best?
#
# Which groups are easy to confuse?

# TODO: build a pipeline
pipeline = Pipeline([
    ('vectorizer', ...),
    ('classifier', KNeighborsClassifier(...))
])

# train, evaluate
pipeline.fit(train.data, train.target)
test_predictions = pipeline.predict(test.data)

# report results
print(sklearn.metrics.classification_report(test.target, test_predictions, target_names=test.target_names))
disp = ConfusionMatrixDisplay.from_predictions(test_predictions, test.target, display_labels=train.target_names, xticks_rotation='vertical')
plt.show()

In [ ]:
# Copy the above code, try different vectorizers and parameters

# 3. Document classification with Naive Bayes

Recall that Naive Bayes uses the Bayes theorem to model the conditional distribution of labels given the features of a sample:
$$P(y | x_1, ..., x_d) = \frac{P(y)P(x_1, ..., x_d | y)}{P(x_1, ..., x_d)}$$

The denominator $P(x_1, ..., x_d)$ is just a constant given by our training data.

The "Naive" in Naive Bayes means that we naively assume the individual features to be (conditionally) pair-wise independent:
$$P(x_1, ..., x_d | y) \approx P(x_1|y)P(x_2|y)\cdots P(x_d|y)$$

Training an NB classifier involves estimating the probabilities $P(y)$ and $P(x_i|y)$ from the training data.

Classification is then done by simply choosing a label $\hat{y}$ that maximizes:
$$\hat{y} = \text{argmax}_y P(y)P(x_1|y)P(x_2|y)\cdots P(x_d|y)$$
for a given sample $x = (x_1, x_2, ..., x_d)$.

# Choosing features and modelling their distributions
$P(y)$ is simple to estimate from the proportion of each label in our training data. However, what features $x_i$ do we choose from our documents? How do we model their distributions $P(x_i|y)$? We have several options, try some of them:

### 1. Simplest: Bernoulli Naive Bayes
The simplest approach is to select a set of words (or the entire vocabulary) and for each word ask: "Is this word present in the document?" Representing the answer as 0/1, we obtain [Bernoulli distributed](https://en.wikipedia.org/wiki/Bernoulli_distribution) features.

With these features, we can train a [`BernoulliNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html).
Hint: The `CountVectorizer` from `sklearn.feature_extraction.text` can be used to get such features. (BernoulliNB simply checks if a feature is nonzero and treats it as binary)

### 2. Better: Multinomial Naive Bayes
Instead of a binary flag, we can model the word counts from `CountVectorizer` using the [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution) with [`MultinomialNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html). While the distribution is defined for discrete values, the TF-IDF vectors can also be used with `MultinomialNB`. Which feature vectors perform better?

### 3. Another possible option: Gaussian Naive Bayes
The [`GaussianNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) assumes the features to be distributed by the [Gaussian distribution](https://en.wikipedia.org/wiki/Normal_distribution). This means that it works with any real-valued features. We can therefore use it with word counts, TF-IDF vectors, as well as word embeddings.
(However, the implementation does not like sparse representations.)

In [ ]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

# Build a Naive Bayes classifier
# First vectorize the documents. Try different Naive Bayes implementations and vectorizers:
#
# Compatible combinations:
#  - BernoulliNB + CountVectorizer
#  - MultinomialNB + CountVectorizer / TfidfVectorizer
#  - GaussianNB + EmbeddingVectorizer / IdfWeightedEmbeddingVectorizer
#
# Try different parameters for the classifier

# TODO: build a pipeline
pipeline = Pipeline([
    ('vectorizer', ...),
    ('classifier', ...),
])

# train, evaluate
pipeline.fit(train.data, train.target)
test_predictions = pipeline.predict(test.data)

# report results
print(sklearn.metrics.classification_report(test.target, test_predictions, target_names=test.target_names))
disp = ConfusionMatrixDisplay.from_predictions(test_predictions, test.target, display_labels=train.target_names, xticks_rotation='vertical')
plt.show()

In [ ]:
# Copy the above code, try different vectorizers, classifiers, and parameters